In [1]:
from models import EEGNet, DeepConvNet
from dataloader import read_bci_data
import torch
import torch.nn as nn
from torch.utils.data import DataLoader,TensorDataset
import torch.optim as optim

In [2]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


In [3]:
def seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True

seed(500)

In [4]:
## Load data
train_data, train_label, test_data, test_label = read_bci_data()

#Input Standardization
#train_data = (train_data - train_data.mean())/(train_data.std())
#test_data = (test_data - test_data.mean())/(test_data.std())


train_data = torch.Tensor(train_data)
train_label = torch.Tensor(train_label)

test_data = torch.Tensor(test_data)
test_label = torch.Tensor(test_label)

train = TensorDataset(train_data, train_label)
train_dataload = DataLoader(train,batch_size=64, shuffle=True)

test = TensorDataset(test_data, test_label)
test_dataload = DataLoader(test,batch_size=64, shuffle=False)

In [ ]:
best_acc=0

for i in [0.004, 0.005, 0.006]: #learning rate 
    for j in [0.003, 0.004, 0.005]: #weight decay
        for k in [250, 300, 350]: # training epoch
            
            model=EEGNet()
            model.to(device)
            optimizer = optim.Adam(model.parameters(), lr=i, weight_decay=j)
            loss_function = torch.nn.CrossEntropyLoss()

            for epoch in range(k):
    
                train_loss = 0.0
                correct = 0
    
                for iteration, (data, label) in enumerate(train_dataload):
                    label = label.type(torch.LongTensor)
                    data = data.to(device)
                    label = label.to(device)
        
                    # clear gradient
                    optimizer.zero_grad()

                    # Forward propagation
                    output = model(data)
        
                    loss = loss_function(output, label)
        
                    # Calculate gradients
                    loss.backward()

                    # Update parameters
                    optimizer.step()
        
                    predicted = torch.max(output.data, 1)[1]
                    correct += (predicted == label).sum().item()
                    train_loss += loss.item()
    
            print("learning rate: {}".format(i))
            print("weight decay: {}".format(j))
            print("epoch: {}".format(k))
            
            model.eval()
            with torch.no_grad():
    
                correct = 0
    
                for iteration, (data, label) in enumerate(test_dataload):
        
                    label = label.type(torch.LongTensor)
                    data = data.to(device)
                    label = label.to(device)
        
                    output = model(data)
        
                    predicted = torch.max(output.data, 1)[1]
                    correct += (predicted == label).sum().item()
    
                acc = correct/1080
                print("EEG Accuracy: ")
                print(acc)
                print("--------------------------------")
                if(acc>best_acc):
                    best_acc=acc
                if(acc>=0.87):
                    torch.save(model, 'EEG_model.pt')
                    break
print("Best accuracy: ")
print(best_acc)

In [10]:
model = torch.load('EEG_model.pt')
model.eval()

with torch.no_grad():
    
                correct = 0
    
                for iteration, (data, label) in enumerate(test_dataload):
        
                    label = label.type(torch.LongTensor)
                    data = data.to(device)
                    label = label.to(device)
        
                    output = model(data)
        
                    predicted = torch.max(output.data, 1)[1]
                    correct += (predicted == label).sum().item()
    
                acc = correct/1080
                print("EEG Accuracy: ")
                print(acc)


EEG Accuracy: 
0.8712962962962963
